## 11.2:   Linear Interpolation

Those taking or who have taken thermodynamics know about using steam tables and other property tables.

In [ ]:
import numpy as np

TC = np.array([ 0.01,  5., 10., 15., 20.,
               25.  , 30., 35., 40., 45.,
               50.  , 55., 60., 65., 70.,
               75.  , 80., 85., 90., 95. ])

P_sat = np.array([ 0.6113,  0.8721,  1.2276,  1.7051,  2.339,
                   3.169 ,  4.246 ,  5.628 ,  7.384 ,  9.593,
                  12.349 , 15.758 , 19.940 , 25.03  , 31.19 ,
                  38.58  , 47.39  , 57.83  , 70.14  , 84.55  ])

assert TC.size == P_sat.size, "Data size mismatch between TC and P_sat"

print("Saturation Pressure [kPa] versus Temperature [C]")
print()
print("Temperature    Pressure")
print("-----------    --------")
for T, P in zip(TC, P_sat):
    string  = "   "
    string += '{:5.2f}'.format(T)
    string += "       "
    string += '{:7.4f}'.format(P)
    print(string)

Suppose you want the saturation pressure at 21.5 C.  Since this temperature is not explicitly in the table, the usual approach is to use linear interpolation.  The linear interpolation formula can be derived by taking ratios of the independent and dependent variables.
<br><br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
$
{\large \frac{T \ - \ T_1}{T_2 \ - \ T_1}} \ \ = \ \ {\large \frac{P \ - \ P_1}{P_2 \ - \ P_1}}
$
&nbsp; &nbsp; &nbsp; &nbsp;
or, for &nbsp; 
$
T_{i-1} \leq T \leq T_i
$
: &nbsp; &nbsp; &nbsp; &nbsp;
$
{\large \frac{T \ - \ T_{i-1}}{T_i \ - \ T_{i-1}}} \ \ = \ \ {\large \frac{P \ - \ P_{i-1}}{P_i \ - \ P_{i-1}}}
$
<br><br>
so that<br><br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
$
P \ \ = \ \ P_{i-1} \ + \ (T \ - \ T_{i-1}) \! \left( {\large \frac{P_i \ - \ P_{i-1}}{T_i \ - \ T_{i-1}}} \right)
$

In [ ]:
def linearly_interpolate_Psat(T):
    """Linear interpolation of Psat vs T data.
    
    Uses TC and P_sat arrays from previous cell (so that cell must be run first)
    """

    assert T >= TC[0] and T <= TC[-1], "Temperature is out of range"

    for i, t in enumerate(TC):    # find the input temperature in the list
        if (t == T):              # the input T has a known P
            return P_sat[i]
        elif (t > T):             # t has gone above T, so this t is T_2 = TC[i]; then T_1 = TC[i-1]
            P = P_sat[i-1] + (T - TC[i-1]) * ( (P_sat[i] - P_sat[i-1]) / (TC[i] - TC[i-1]) )
            return P
        else:                     # t < T so keep looking for when t crosses over T
            continue              # continue loop
    assert False, "If we get here, our algorithm failed or the TC array is not monotonic increasing"

T = 21.5
P = linearly_interpolate_Psat(T)
print("T [C] = ", T, "and Psat [kPa] = ", P)

But if we plot P_sat versus TC the curve is clearly not linear (although it does look reasonably linear between 15 and 30 degrees C).

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1)
plt.plot(TC, P_sat, 'r-', linewidth = 2.0)
plt.xlabel("Temperature, T [K]", fontsize=12, labelpad=10)
plt.ylabel("Saturation Pressure, Psat [kPa]", fontsize=12, labelpad=10)
title_string  = "Non-linear saturation pressure dependence on temperature"
plt.title(title_string)
plt.show()

fig = plt.figure(2)
plt.plot(TC, P_sat, 'r-', linewidth = 2.0)
plt.xlabel("Temperature, T [K]", fontsize=12, labelpad=10)
plt.ylabel("Saturation Pressure, Psat [kPa]", fontsize=12, labelpad=10)

# zoom in on region of interest
plt.xlim(15.0, 30.0)
plt.ylim( 1.7,  4.3)
title_string  = "Non-linear saturation pressure dependence on temperature\n"
title_string += "zoomed in on interval of interest"

plt.title(title_string)
plt.show()


To do better we can use Lagrange interpolation to calculate a higher order fit.